In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from inxss import SirenNet

from sklearn.model_selection import train_test_split

from lightning.pytorch.loggers import TensorBoardLogger

In [2]:
from inxss import SpectrumDataset, SpecNeuralRepr
from torch.utils.data import DataLoader

In [3]:
spec_dataset = SpectrumDataset(
        '/pscratch/sd/z/zhantao/inxs_steering/SpinW_data/summarized_smoothed_AFM_data_2023Sep13.pt',
        num_wq=10000
    )

In [4]:
train_idx, val_test_idx = train_test_split(np.arange(len(spec_dataset)), test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(val_test_idx, test_size=0.5, random_state=42)

train_loader = DataLoader([spec_dataset[i] for i in train_idx], batch_size=10, shuffle=True)
val_loader = DataLoader([spec_dataset[i] for i in val_idx], batch_size=10, shuffle=False)
test_loader = DataLoader([spec_dataset[i] for i in test_idx], batch_size=10, shuffle=False)

In [5]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, TQDMProgressBar

model = SpecNeuralRepr(
    scale_dict={
            'J' : [(20, 40), (0, 0.5)], 
            'Jp': [(-5,  5), (0, 0.5)], 
            'w' : [(0, 250), (0, 0.5)]
        }
)

In [6]:
checkpoint_callback = ModelCheckpoint(
    save_on_train_epoch_end=False, save_last=True, save_top_k=1, monitor="val_loss"
)

logger = TensorBoardLogger(save_dir='/pscratch/sd/z/zhantao/inxs_steering')

trainer = L.Trainer(
    max_epochs=10000, accelerator="gpu",
    callbacks=[TQDMProgressBar(refresh_rate=10), checkpoint_callback],
    logger=logger, log_every_n_steps=1, devices=1,
    enable_checkpointing=True,
    default_root_dir='/pscratch/sd/z/zhantao/inxs_steering'
)

trainer.fit(model, train_loader, val_loader)

Epoch 8304: 100%|██████████| 48/48 [00:01<00:00, 27.74it/s, v_num=1.49e+7]
Validation: 0it [00:00, ?it/s]
Epoch 8305: 100%|██████████| 48/48 [00:01<00:00, 31.30it/s, v_num=1.49e+7]
Validation: 0it [00:00, ?it/s]
Epoch 8306: 100%|██████████| 48/48 [00:01<00:00, 28.95it/s, v_num=1.49e+7]
Validation: 0it [00:00, ?it/s]
Epoch 8307: 100%|██████████| 48/48 [00:01<00:00, 30.70it/s, v_num=1.49e+7]
Validation: 0it [00:00, ?it/s]
Epoch 8308: 100%|██████████| 48/48 [00:01<00:00, 27.16it/s, v_num=1.49e+7]
Validation: 0it [00:00, ?it/s]
Epoch 8309: 100%|██████████| 48/48 [00:01<00:00, 27.06it/s, v_num=1.49e+7]
Validation: 0it [00:00, ?it/s]
Epoch 8310:  21%|██        | 10/48 [00:00<00:01, 32.52it/s, v_num=1.49e+7]

/pscratch/sd/z/zhantao/conda/inxs/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
